In [10]:
import click
from pysr import pysr, best, get_hof
import pandas as pd
import os 
from dispersant_screener.definitions import FEATURES
from sklearn.preprocessing import StandardScaler
import numpy as np 

DATADIR = '../data'

In [11]:
def load_data(n_samples, label_scaling: bool = False):
    """Take in Brian's data and spit out some numpy arrays for the PAL"""
    df_full_factorial_feat = pd.read_csv(os.path.join(DATADIR, 'new_features_full_random.csv'))[FEATURES].values
    a2 = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values
    deltaGMax = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values  # pylint:disable=unused-variable
    gibbs = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_deltaG.csv'))['deltaGmin'].values * (-1)
    gibbs_max = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['deltaGmax'].values
    force_max = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_fit2.csv'))['F_repel_max'].values  # pylint:disable=unused-variable
    rg = pd.read_csv(os.path.join(DATADIR, 'rg_results.csv'))['Rg'].values
    y = np.hstack([rg.reshape(-1, 1), gibbs.reshape(-1, 1), gibbs_max.reshape(-1, 1)])
    assert len(df_full_factorial_feat) == len(a2) == len(gibbs) == len(y)

    feat_scaler = StandardScaler()
    X = feat_scaler.fit_transform(df_full_factorial_feat)

    if label_scaling:
        label_scaler = MinMaxScaler()
        y = label_scaler.fit_transform(y)

    #greedy_indices = get_maxmin_samples(X, n_samples)

    return X, y#, greedy_indice

In [12]:
X, y = load_data(1)

In [ ]:
equations = pysr(
    X,
    y[:, 0],
    niterations=100,
    select_k_features=15,
    binary_operators=["plus", "mult"],
    unary_operators=["cos", "exp", "sin", "neg", "square", "cube", "logm", "sqrtm"],
    equation_file=f"hall_of_fame.csv",
)

Using features [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Running on julia -O3 -p 4 /tmp/.runfile_76988024285508687300.jl
Started!

Cycles per second: 2.400e+04
Hall of Fame:
-----------------------------------------
Complexity  MSE        Score     Equation
0           5.723e-02  0.000e+00  2.150160
1           5.723e-02  5.960e-08  2.150166


Cycles per second: 1.230e+04
Hall of Fame:
-----------------------------------------
Complexity  MSE        Score     Equation
0           5.723e-02  0.000e+00  2.150160
1           5.723e-02  5.960e-08  2.150166


Cycles per second: 2.220e+04
Hall of Fame:
-----------------------------------------
Complexity  MSE        Score     Equation
0           5.723e-02  0.000e+00  2.150160
1           5.723e-02  5.960e-08  2.150166


Cycles per second: 2.120e+04
Hall of Fame:
-----------------------------------------
Complexity  MSE        Score     Equation
0           5.723e-02  0.000e+00  2.150160
1           5.723e-02  5.960e-08  2.150166


Cycles 